<a href="https://colab.research.google.com/github/ZotovNV/Resume_for_chek/blob/main/%D0%9F%D1%80%D0%BE%D0%B4%D0%B2%D0%B8%D0%BD%D1%83%D1%82%D1%8B%D0%B9_Pandas_%D0%94%D0%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1:\
Для датафрейма log из материалов занятия создайте столбец source_type по правилам: \

если источник traffic_source равен Yandex или Google, то в source_type ставится organic; \
для источников paid и email из России ставим ad; \
для источников paid и email не из России ставим other; \
все остальные варианты берём из traffic_source без изменений. \

In [ ]:
import pandas as pd
from zipfile import ZipFile

url = 'C:/Users/User/Desktop/Никита обучение/доп_материалы_к_пандас_1_и_2.zip'
zf = ZipFile(url)
visit_log = pd.read_csv(zf.open('visit_log.csv', mode = 'r'), sep = ';')

def source_type(row):
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        return 'organic'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email') and row['region'] == 'Russia':
        return 'ad'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email') and row['region'] != 'Russia':
        return 'other'
    else:
        return row['traffic_source']

visit_log['source_type'] = visit_log.apply(source_type, axis=1)
visit_log.head()

Задание 2: \
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм. \
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [ ]:
import pandas as pd
from zipfile import ZipFile

url = 'C:/Users/User/Desktop/Никита обучение/доп_материалы_к_пандас_1_и_2.zip'
zf = ZipFile(url)
urls = pd.read_csv(zf.open('URLs.txt', mode = 'r'))


news_urls = urls[urls['url'].str.contains('/\d{8}-')]
news_urls.info()

Задание 3:\
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd
from zipfile import ZipFile

url = 'C:/Users/User/Desktop/Никита обучение/ml-25m.zip'
zf = ZipFile(url)
ratings = pd.read_csv(zf.open('ml-25m/ratings.csv', mode = 'r'))

# Сгруппировали по id пользователя и посчитали кол-во оценок.
user_ratings_count = ratings.groupby('userId').count()['rating']
# user_ratings_count.head()
# Сформировали список с id пользователей где кол-во выставленных оценок больше 100.
active_users = user_ratings_count[user_ratings_count > 100].index.tolist()
# print(active_users)
# Отсортировали фрейм по списку пользователей выставивших больше 100 оценок
active_ratings = ratings[ratings['userId'].isin(active_users)]
# active_ratings.head()
# Сгруппировали по id пользователя, нашли самое большое и самое маленькое значение timestamp.
user_lifetime = active_ratings.groupby('userId').agg({'timestamp': ['max', 'min']})
# Посчитали среднее время жизни пользователей.
# user_lifetime.head()
mean_lifetime = (user_lifetime['timestamp']['max'] - user_lifetime['timestamp']['min']).mean()
print(mean_lifetime)

# результат 42697549.00773351

Задание 4:
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»). Нужно сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
аналогичную таблицу по типам выручки с указанием адреса клиента.
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
import pandas as pd

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
revenue = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
revenue.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [ ]:
revenue_with_address = revenue.merge(client_base, on='client_id', how='left')
revenue_with_address.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
